In [19]:
command = """az account list --query "[?name=='AAFC VSE Benefit'].id" -o tsv"""
subscription_id = !{command}
subscription_id = subscription_id[0]

In [20]:
from pathlib import Path
template_dir = Path("template")
assert template_dir.exists()

In [21]:
output_dir = Path("rendered")

In [22]:
# ensure "rendered" is in gitignore exactly once
with open(".gitignore", "r") as f:
    gitignore = f.read()
    gitignore = gitignore.strip()
    gitignore = gitignore.split("\n")
    gitignore = [line.strip() for line in gitignore]
    gitignore = [line for line in gitignore if line]
    gitignore = set(gitignore)
    gitignore.add("rendered")
    gitignore = sorted(gitignore)
    gitignore = "\n".join(gitignore)
    gitignore += "\n"
    
    # write the new contents
with open(".gitignore", "w") as f:
    f.write(gitignore)

In [23]:
warning_header = """
# THIS FILE IS MANAGED BY AUTOMATION!
# IF YOU WANT TO MAKE CHANGES, EDIT THE TEMPLATE AND RE-RUN THE AUTOMATION
# GO TO `iac-kubernetes/manifest_templates.ipynb` TO FIND THE AUTOMATION
""".strip()

In [24]:
variables = {
    "subscription_id": subscription_id,
    "warning_header": warning_header,
}

In [25]:
%pip install -q jinja2

Note: you may need to restart the kernel to use updated packages.


In [26]:
from jinja2 import Environment, FileSystemLoader, StrictUndefined
env = Environment(
    loader=FileSystemLoader(template_dir), 
    undefined=StrictUndefined,
)

In [27]:
# perform the rendering
for template_path in template_dir.glob("**/*"):
    output_path = output_dir / template_path.relative_to(template_dir)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"To {output_path} from {template_path}")
    template = env.get_template(str(template_path.relative_to(template_dir)))
    output = template.render(**variables)
    with open(output_path, "w") as f:
        f.write(output)

To rendered\main.tf from template\main.tf
